# 如何在 Colab 安裝 Darknet 框架訓練 YOLO v3 物件辨識並且最佳化 Colab 的訓練流程

此範例程式碼為文章「如何在 Colab 安裝 Darknet 框架訓練 YOLO v3 物件辨識並且最佳化 Colab 的訓練流程」的範例程式碼。

這篇文章會教你：
* 利用 Colab 128G RAM GPU 來訓練你的 Yolo3 模型
* 將 Colab 設定成可以運用在實際專案的訓練環境
* 快速掛載本機電腦上的檔案到 Colab 環境中
* 事先編譯所有需要的檔案，每次開啟 Colab 後即可立刻進行訓練
* 將訓練好的 weight 檔案自動同步回本機電腦，避免檔案遺失

## Step 2：連接你的 Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive') # 請點擊網址，選擇 Google 帳號登入，然後將授權碼貼回輸入框中

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!ln -fs /content/gdrive/My\ Drive /app

ln: failed to create symbolic link '/app/My Drive': Input/output error


## 步驟 4：下載並編譯 Darknet（只需要執行一次）

這個步驟只需要執行一次，執行過程我們會把執行檔複製到 Google Drive。之後要跑程式時，只要把執行檔複製回 Colab 環境就可以使用了。

### 4.1 下載 cuDNN 檔案

1. 申請 Nvidia 帳號，申請網址為 http://bit.ly/2qfpOPj
2. 下載 `cudnn-linux-x86_64-8.7.0.84_cuda11-archive.tar.xz`，下載網址為
下載 cuDNN 檔案。下載網址為：http://bit.ly/2qfpOPj
3. 將下載的檔案 `cudnn-linux-x86_64-8.7.0.84_cuda11-archive.tar.xz` 放到 google drive 的 `darknet/cuDNN/` 目錄下。

In [3]:
!tar -Jxvf /app/darknet/cuDNN/cudnn-linux-x86_64-8.7.0.84_cuda11-archive.tar.xz -C /usr/local/
#!dpkg -i /app/cudnn/cudnn-local-repo-ubuntu2004-8.9.1.23_1.0-1_amd64.deb
!chmod a+r /usr/local/cuda/include/cudnn.h

# 檢查是否安裝成功
!cat /usr/local/include/cudnn_version.h | grep CUDNN_MAJOR -A 2

cudnn-linux-x86_64-8.7.0.84_cuda11-archive/
cudnn-linux-x86_64-8.7.0.84_cuda11-archive/lib/
cudnn-linux-x86_64-8.7.0.84_cuda11-archive/lib/libcudnn_adv_infer_static.a
cudnn-linux-x86_64-8.7.0.84_cuda11-archive/lib/libcudnn_adv_infer_static_v8.a
cudnn-linux-x86_64-8.7.0.84_cuda11-archive/lib/libcudnn_adv_train_static.a
cudnn-linux-x86_64-8.7.0.84_cuda11-archive/lib/libcudnn_adv_train_static_v8.a
cudnn-linux-x86_64-8.7.0.84_cuda11-archive/lib/libcudnn_cnn_infer_static.a
cudnn-linux-x86_64-8.7.0.84_cuda11-archive/lib/libcudnn_cnn_infer_static_v8.a
cudnn-linux-x86_64-8.7.0.84_cuda11-archive/lib/libcudnn_cnn_train_static.a
cudnn-linux-x86_64-8.7.0.84_cuda11-archive/lib/libcudnn_cnn_train_static_v8.a
cudnn-linux-x86_64-8.7.0.84_cuda11-archive/lib/libcudnn_ops_infer_static.a
cudnn-linux-x86_64-8.7.0.84_cuda11-archive/lib/libcudnn_ops_infer_static_v8.a
cudnn-linux-x86_64-8.7.0.84_cuda11-archive/lib/libcudnn_ops_train_static.a
cudnn-linux-x86_64-8.7.0.84_cuda11-archive/lib/libcudnn_ops_train_st

In [4]:
# 檢查是否安裝成功
!ls /usr/include/cudnn*

/usr/include/cudnn_adv_infer.h	/usr/include/cudnn.h
/usr/include/cudnn_adv_train.h	/usr/include/cudnn_ops_infer.h
/usr/include/cudnn_backend.h	/usr/include/cudnn_ops_train.h
/usr/include/cudnn_cnn_infer.h	/usr/include/cudnn_version.h
/usr/include/cudnn_cnn_train.h


In [5]:
#!find / -name cudnn.h -print
#!cp /usr/include/cudnn_version.h /content
!cat /usr/include/cudnn_version.h | grep CUDNN_MAJOR -A 8

#define CUDNN_MAJOR 8
#define CUDNN_MINOR 9
#define CUDNN_PATCHLEVEL 0

#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

/* cannot use constexpr here since this is a C-only file */
/* Below is the max SM version this cuDNN library is aware of and supports natively */

#define CUDNN_MAX_SM_MAJOR_NUMBER 9
#define CUDNN_MAX_SM_MINOR_NUMBER 0
#define CUDNN_MAX_DEVICE_VERSION (CUDNN_MAX_SM_MAJOR_NUMBER * 100 + CUDNN_MAX_SM_MINOR_NUMBER * 10)



### 4.2 下載並編譯 Darknet

In [6]:
!apt install libopencv-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopencv-dev is already the newest version (4.5.4+dfsg-9ubuntu4).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [7]:
!apt-get install g++
#!pkg-config --modversion opencv
#!pkg-config --cflags opencv
#!find / -name opencv.pc -print

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [17]:
%cd /content/
!rm -rf darknet_source
!git clone https://github.com/kriyeng/darknet/ darknet_source
%cd darknet_source
#!cp /content/darknet_source/Makefile .
# 修改 Darknet 設定，符合 Colab 環境
#!sed -i "s/GPU=0/GPU=1/g" Makefile
#!sed -i "s/CUDNN=0/CUDNN=1/g" Makefile
!sed -i "s/OPENCV=0/OPENCV=1/g" Makefile

/content
Cloning into 'darknet_source'...
remote: Enumerating objects: 10068, done.
remote: Total 10068 (delta 0), reused 0 (delta 0), pack-reused 10068
Receiving objects: 100% (10068/10068), 10.14 MiB | 17.63 MiB/s, done.
Resolving deltas: 100% (6715/6715), done.
/content/darknet_source


In [22]:
!cat Makefile | grep GPU
!cat Makefile | grep CUDNN

GPU=0
ifeq ($(GPU), 1) 
COMMON+= -DGPU -I/usr/local/cuda/include/
CFLAGS+= -DGPU
ifeq ($(GPU), 1) 
CUDNN=0
ifeq ($(CUDNN), 1) 
COMMON+= -DCUDNN 
CFLAGS+= -DCUDNN


In [18]:
!cp /usr/include/cudnn* /usr/local/cuda/include

In [20]:
%cd /content/darknet_source
# 編譯
!make clean
!make

/content/darknet_source
rm -rf ./obj/gemm.o ./obj/utils.o ./obj/cuda.o ./obj/deconvolutional_layer.o ./obj/convolutional_layer.o ./obj/list.o ./obj/image.o ./obj/activations.o ./obj/im2col.o ./obj/col2im.o ./obj/blas.o ./obj/crop_layer.o ./obj/dropout_layer.o ./obj/maxpool_layer.o ./obj/softmax_layer.o ./obj/data.o ./obj/matrix.o ./obj/network.o ./obj/connected_layer.o ./obj/cost_layer.o ./obj/parser.o ./obj/option_list.o ./obj/detection_layer.o ./obj/route_layer.o ./obj/upsample_layer.o ./obj/box.o ./obj/normalization_layer.o ./obj/avgpool_layer.o ./obj/layer.o ./obj/local_layer.o ./obj/shortcut_layer.o ./obj/logistic_layer.o ./obj/activation_layer.o ./obj/rnn_layer.o ./obj/gru_layer.o ./obj/crnn_layer.o ./obj/demo.o ./obj/batchnorm_layer.o ./obj/region_layer.o ./obj/reorg_layer.o ./obj/tree.o ./obj/lstm_layer.o ./obj/l2norm_layer.o ./obj/yolo_layer.o ./obj/iseg_layer.o ./obj/image_opencv.o libdarknet.so libdarknet.a darknet ./obj/captcha.o ./obj/lsd.o ./obj/super.o ./obj/art.o ./obj/

In [11]:
# 將編譯完成的執行檔複製回 Google Drive
!cp ./darknet /app/darknet
!cp ./darknet /content/darknet
%cd /content

cp: cannot stat './darknet': No such file or directory
cp: cannot stat './darknet': No such file or directory
/content


# 測試 Darknet 是否正常安裝

我們用 yolo 官方預先訓練好的模型來偵測範例圖片。

In [12]:
%cd /content/darknet_source
!wget https://pjreddie.com/media/files/yolov3.weights
!/content/darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg -dont-show



/content/darknet_source
--2023-10-01 04:20:08--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  69.0MB/s    in 3.7s    

2023-10-01 04:20:12 (63.1 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]

/bin/bash: line 1: /content/darknet: No such file or directory


In [13]:
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread('predictions.jpg', cv2.IMREAD_UNCHANGED)
cv2_imshow(img)

AttributeError: ignored